<h1><center><font color='Black'>Data Handling</font></center></h1>

In [ ]:
#---------Imports--------#
import pandas as pd
from datetime import date
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

todays_date = date.today() #Get Today'S Date
curryear = todays_date.year
swipe = curryear - 2014 #Start From Curr Year To 2014

In [ ]:
df = pd.read_csv('Data/df.csv', encoding='utf-8-sig' )
df2 = pd.read_csv('Data/df2.csv', encoding='utf-8-sig')

In the second DataFrame we have several regions names in the same row we want to separate them into a separate rows 

In [ ]:
ids = df2["Zone Number"]
df2=df2[ids.isin(ids[ids.duplicated()])].sort_values("Zone Number")

df2 = df2.groupby('Zone Number').agg({'Zone Name': ','.join, 'Defense Time':'first' }).reset_index()
df2 = df2.copy()

Change the type of the date and the time into type of DateTime and create a new column with full DateTime contains the time and the date 

In [ ]:
df["Date"] = pd.to_datetime(df['Date'],format= '%d.%m.%Y')

In [ ]:
df['Datetime'] = pd.to_datetime(df['Date'].apply(str)+' '+df['Time'])

The city of Ashdod is divided into several areas, so we want to turn all those areas into one area called Ashdod

In [ ]:
df.loc[(df['Date'] > '2015') & (df["Zone"].str.contains('אשדוד')),'Zone']='אשדוד'

Since we have several regions names in the same row we want to separate them into a separate rows 

In [ ]:
df['Zone'] = df['Zone'].str.split(',')
df = df.explode('Zone')

After splitting all the areas into separate rows, we will now convert each area to all the cities in the same area, according to the names in the second DataFrame

In [ ]:
for index, row in df2.iterrows():
    df.loc[df['Zone'].str.contains(row['Zone Number']), 'Zone'] = row['Zone Name']
    df.replace(row['Zone Number'] , row['Zone Name'] ,inplace=True)

There are alarms that appear in our data called "test" so we want to get rid of them, in addition because we made all the areas into the names of the cities in the same area, now we will split all the cities into separate rows

In [ ]:
df = df[~df['Zone'].isin(['בדיקה','בדיקה ידנית 1'])] 
df.reset_index(drop=True, inplace=True)

df['Zone'] = df['Zone'].str.split(',')
df = df.explode('Zone')

There are alarms that contain only some of the names of certain cities, so we will make any alarm that contains only some of the names, into the names of the cities themselves

In [ ]:
megaCities = ['באר שבע','אשדוד','ירושלים','תל אביב','נתניה','הרצליה',
'חדרה','אשקלון','ראשון לציון','ניר עם','רמת גן','פתח תקווה','בני ברק' , 'חולון','רחובות']

for index in megaCities:
   df.loc[df['Zone'].str.contains(index), 'Zone'] = index 

In [ ]:
#Drop Duplicates
df.drop_duplicates(inplace=True)

Now the DataFrame is ready , We can move to the next step, to EDA.

In [ ]:
df

In [ ]:
df2.to_csv("Data/df2.csv", encoding='utf-8-sig' ,index=False )
df.to_csv("Data/df.csv", encoding='utf-8-sig' , index=False)